In [7]:
#从服务器上选择一款泰坦，当前是选择了第0块
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
import os
gpu_id='0'
def init_env():
    os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
    os.environ['CUDA_VISIBLE_DEVICES']=gpu_id    
    config=tf.ConfigProto()
    config.gpu_options.allow_growth=True
    set_session(tf.Session(config=config))
init_env()

## 将热点预测模块进行封装

In [13]:
import pandas as pd
import numpy as np
import json
import os
import csv
import pickle
import jieba
import re,time
from phrase_divided import *
from sklearn.decomposition import LatentDirichletAllocation
#导入PCA算法库
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from textrank4zh import TextRank4Keyword, TextRank4Sentence
from sklearn import ensemble

from sklearn.metrics import mean_squared_error #均方误差
from sklearn.metrics import mean_absolute_error #平方绝对误差
from sklearn.metrics import r2_score#R square
import math
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn import neighbors
from sklearn import ensemble
from sklearn.externals import joblib

import time
import random
import os
import re
import pandas as pd
import numpy as np
from datetime import datetime
from elasticsearch import Elasticsearch

In [14]:
class esPaperRetrieval():
    '''
    根据论文检索需求进行功能的微调
    '''
    _instance = None
    _first_init = True
    def __new__(cls, *args, **kw):
        if not cls._instance:
            cls._instance = super(esPaperRetrieval, cls).__new__(cls)  
        return cls._instance
    
    def __init__(self, host, port):
        '''
        使用ES进行论文检索 指定host、port以及专利index之后进行检索
        '''
        super(esPaperRetrieval, self).__init__()
        self.es = Elasticsearch(hosts=host, port=port, timeout=30, max_retries=10, retry_on_timeout=True)
        self.indexName = 'paper-hot-index'

    def do_search(self,summaryQuery,fromDate, toDate, volume):
        '''
        do_search方法执行具体检索过程
        titleQuery 为对应标题检索词
        kwordQuery 为对应关键字检索词
        summaryQuery 为对应摘要检索词
        上述三个词为or模式 可以出现 可以不出现
        pubQuery 为对应文摘关键词 这个关键词必定匹配
        fromDate 为检索字段起始日期 toDate为终结日期 日期格式 yyyy-mm-dd
        volume为每次检索返回的数目
        '''
        queryBody = {
          "query": {
            "bool": {
              "should": [
                {
                  "term": {
                    "P_Summary": summaryQuery
                  }
                }
              ],
              "filter": [
                {
                  "range": {
                    "P_year": {
                      "gt": fromDate,
                      "lt": toDate
                    }
                  }
                }
               
              ]
            }
          },
            "from": 0,
            "size": volume,
            "sort": [],
            "aggs": {}
        }
        print(queryBody)
        result = self.es.search(index=self.indexName, body=queryBody)
        return result

    def format_search(self, result):
        '''
        format_search方法对检索结果进行格式化 构建符合要求的字段进行返回
        输入result为检索结果 提取其中的检索结果进行后处理
        使用ES检索后得到的结果中result['hits']['hits']为数组格式数据
        其中每一个元素为一个dict 对应部分字段
        '''
        docs = result['hits']['hits']
        docs = [i['_source'] for i in docs]
        targetKeyList = 'P_Title, P_Author, P_Publication, P_Organ, P_year, P_Keyword, \
    P_Summary, P_Keyword_seg, P_Title_seg,\
    P_Summary_seg, P_URL, P_Fields, P_Fields_two,P_References, P_Pagecount, P_Page, P_Language,\
    P_Download_num, P_Citation_num,P_Vector,P_Volume, P_Issue,P_Issn,P_Isbn, P_Doi,\
    P_Red1, P_Red2, P_Red3, P_Red4, P_Red5'
        targetKeyList = [i.strip() for i in targetKeyList.split(',')]
        dict_filter_by_keys = lambda d: {k: d[k] for k in targetKeyList}
        dict_filter_text = lambda d: {k if not k == 'text' else 'claim_text': d[k] for k in d}
        dict_filter_id = lambda d: {k if not k == '_id' else 'id': d[k] for k in d}
        docs = (dict_filter_by_keys(doc) for doc in docs)
        docs = [dict_filter_text(doc) for doc in docs]
        return docs
    def Retrieval(self, summaryQuery,fromDate, toDate, volume):
        result = self.do_search(summaryQuery,fromDate, toDate, volume)
        docs = self.format_search(result)
        return docs

In [15]:
eee = esPaperRetrieval(host='10.8.128.205',port=49200,)

In [16]:
aaa = eee.Retrieval(summaryQuery='中医',
              fromDate='2005-01-01',toDate='2018-01-01',volume=10)

{'query': {'bool': {'should': [{'term': {'P_Summary': '中医'}}], 'filter': [{'range': {'P_year': {'gt': '2005-01-01', 'lt': '2018-01-01'}}}]}}, 'from': 0, 'size': 10, 'sort': [], 'aggs': {}}


NotFoundError: NotFoundError(404, 'index_not_found_exception', 'no such index [paper-hot-index]', paper-hot-index, index_or_alias)

In [12]:
aaa

[{'P_Author': '马林林;任长虹;李宁;吉训明;曾现伟;',
  'P_Citation_num': 0,
  'P_Doi': 'none',
  'P_Download_num': 0,
  'P_Fields': '医学',
  'P_Fields_two': 'none',
  'P_Isbn': 'none',
  'P_Issn': 'none',
  'P_Issue': 'none',
  'P_Keyword': '脑缺血; 再灌注损伤; 四肢; 远隔缺血期适应(非MeSH词); 疾病模型;动物',
  'P_Keyword_seg': '脑缺血  再灌注损伤  四肢  远隔缺血期适应(非MeSH词)  疾病模型 动物',
  'P_Language': '中',
  'P_Organ': '山东省潍坊医学院;首都医科大学宣武医院低氧医学研究室;首都医科大学宣武医院神经外科;山东潍坊医学院附属医院神经外科;',
  'P_Page': '313-317',
  'P_Pagecount': 5,
  'P_Publication': '中国现代神经疾病杂志',
  'P_Red1': 'none',
  'P_Red2': 'none',
  'P_Red3': 'none',
  'P_Red4': 'none',
  'P_Red5': 'none',
  'P_References': 'none',
  'P_Summary': '研究背景肢体远隔缺血期适应已成为脑缺血再灌注损伤中的一项治疗策略,然而目前应用的缺血期适应方法为有创性操作技术,限制了其长期应用的可能性,更难实现基础研究向临床应用的转化。鉴于此,若能使无创性肢体远隔缺血期适应与有创性肢体远隔缺血期适应具有同样的脑保护效果,则更有利于临床推广。方法采用线栓法建立大鼠大脑中动脉缺血再灌注损伤模型,分别于缺血开始通过肢体远端气囊加压法或远隔肢体动脉夹闭(缺血)10 min 再灌注10 min(共3 个循环)方法,制备无创和有创缺血期适应动物模型。根据12分评分法进行神经功能评分、TTC染色测定脑梗死体积和脑水肿程度。结果与单纯缺血再灌注损伤组相比,无创和有创缺血期适应组大鼠神经功能评分改善(P=0.041,0.035)、脑水肿体积(P=0.040,0.028)和梗死灶体积